In [2]:
#---------------------------PART 3.1 A* SHORTEST PATH---------------------------------------#
import math

class WeightedGraph:

    def __init__(self,nodes): #nodes is an int
        self.graph=[]
        self.weights={}
        for node in range(nodes):  #created 2D lsit of len nodes
            self.graph.append([])

    def add_node(self,node):
        self.graph[node]=[]

    def add_edge(self, node1, node2, weight):
        if node2 not in self.graph[node1]:
            self.graph[node1].append(node2)
        self.weights[(node1, node2)] = weight

    def get_weights(self, node1, node2):
        if self.are_connected(node1, node2):
            return self.weights[(node1, node2)]

    def are_connected(self, node1, node2):
        for neighbour in self.graph[node1]:
            if neighbour == node2:
                return True
        return False

    def get_neighbors(self, node):
        return self.graph[node]

    def get_number_of_nodes(self,):
        return len(self.graph)
    
    def get_nodes(self,):
        return [i for i in range(len(self.graph))]

In [3]:
#----------------------------------AUXILARY FUNCTIONS-------------------------------------------------#
class MinHeap:
    def __init__(self, data):
        self.items = data #list of 'Item' type objects, which have a key and a value
        self.length = len(data)
        self.build_heap()

        # add a map based on input node
        self.map = {}
        for i in range(self.length):
            self.map[self.items[i].value] = i #create a dictonary of values(weights) to keys (nodes)

    def find_left_index(self,index):
        return 2 * (index + 1) - 1

    def find_right_index(self,index):
        return 2 * (index + 1)

    def find_parent_index(self,index):
        return (index + 1) // 2 - 1  
    
    def sink_down(self, index):
        smallest_known_index = index

        if self.find_left_index(index) < self.length and self.items[self.find_left_index(index)].key < self.items[index].key:
            smallest_known_index = self.find_left_index(index)

        if self.find_right_index(index) < self.length and self.items[self.find_right_index(index)].key < self.items[smallest_known_index].key:
            smallest_known_index = self.find_right_index(index)

        if smallest_known_index != index:
            self.items[index], self.items[smallest_known_index] = self.items[smallest_known_index], self.items[index]
            
            # update map
            self.map[self.items[index].value] = index #indexes were swapped, now fix values (val:new_key/node)
            self.map[self.items[smallest_known_index].value] = smallest_known_index 

            # recursive call
            self.sink_down(smallest_known_index)

    def build_heap(self,):
        for i in range(self.length // 2 - 1, -1, -1):
            self.sink_down(i) 

    def insert(self, node):
        if len(self.items) == self.length:
            self.items.append(node)
        else:
            self.items[self.length] = node #insert node of type 'Item' to items
        self.map[node.value] = self.length #ad val:key to dictionary 'map'
        self.length += 1 #update len
        self.swim_up(self.length - 1) #move new node to correct pos

    def insert_nodes(self, node_list):
        for node in node_list:
            self.insert(node)

    def swim_up(self, index):
        
        while index > 0 and self.items[self.find_parent_index(index)].key < self.items[self.find_parent_index(index)].key:
            #swap values
            self.items[index], self.items[self.find_parent_index(index)] = self.items[self.find_parent_index(index)], self.items[index]
            #update map
            self.map[self.items[index].value] = index
            self.map[self.items[self.find_parent_index(index)].value] = self.find_parent_index(index)
            index = self.find_parent_index(index)

    def get_min(self):
        if len(self.items) > 0:
            return self.items[0]

    def extract_min(self,):
        #xchange
        self.items[0], self.items[self.length - 1] = self.items[self.length - 1], self.items[0]
        #update map
        self.map[self.items[self.length - 1].value] = self.length - 1
        self.map[self.items[0].value] = 0

        min_node = self.items[self.length - 1]
        self.length -= 1
        self.map.pop(min_node.value)
        self.sink_down(0)
        return min_node

    def decrease_key(self, value, new_key):
        if new_key >= self.items[self.map[value]].key:
            return
        index = self.map[value]
        self.items[index].key = new_key
        self.swim_up(index)

    def get_element_from_value(self, value):
        return self.items[self.map[value]]

    def is_empty(self):
        return self.length == 0
    
    def find_item(self, k):

        for i in self.items:
            if i.key == k:
                return True
        
        return False
    
    def __str__(self):
        height = math.ceil(math.log(self.length + 1, 2))
        whitespace = 2 ** height + height
        s = ""
        for i in range(height):
            for j in range(2 ** i - 1, min(2 ** (i + 1) - 1, self.length)):
                s += " " * whitespace
                s += str(self.items[j]) + " "
            s += "\n"
            whitespace = whitespace // 2
        return s


In [4]:
class Item:
    def __init__(self, key, value):
        self.key = key #node 
        self.value = value #f

    
    def __str__(self):
        return "(" + str(self.key) + "," + str(self.value) + ")"

In [5]:
#class Node:
#    def __init__(self, key, f_val, x = 0, y = 0):
#        self.key = key
#        self.value = f_val
#        self.x = x
#        self.y = y
#
#        #self.parent = None
#       # self.child = None




In [6]:
#calculate heuristics forall nodes in the graph
def heuristic(g: WeightedGraph, target_node, x_coor, y_coor):

    h = {}

    for node in range(len(g.graph)):

        x1 = x_coor[node]
        y1 = y_coor[node]

        x2 = x_coor[target_node]
        y2 = y_coor[target_node]

        dist = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)

        h[node] = dist

    return h

In [41]:
def shortest_path(pred, dest, s):

    path = []

    curr = dest

    while pred[curr] != None:

        path = [curr] + path

        curr = pred[curr]

    path = [s] + path

    print(path)

    return path

In [44]:
def A_Star(G: WeightedGraph, source, destination, h): 
    
    graph = G.graph

    #initialize open list and put start/source node in it and its initial f of 0
    open = MinHeap([])
    open.insert(Item(source, 0))

    #initialize closed list
    closed = []

    #initialize predessor dict
    pred = {source: None}

    #initialize all node g and f values to be infinity
    g = {}

    for node in range(len(graph)):
        g[node] = float('inf')

    g[source] = 0

    f = {}
    
    for node in range(len(graph)):
        f[node] = float('inf')


    f[source] = h[source]


    #-------------------------------------------------------


    #search
    while open.length != 0:

        #print(open.map)
        curr_f = open.extract_min() #(node, f)
        curr = curr_f.key

        #-------------------------------------------
       
       
       # print(curr)

        if curr == destination:
            return (pred, shortest_path(pred, destination, source)) 
        
        #iterate through the current nodes neighbors
        for node in graph[curr]:

            new_g = g[curr] + G.get_weights(curr,node)

            if new_g < g[node]:

                pred[node] = curr

                g[node] = new_g
                f[node] = new_g + h[node]

                #if node not in open or closed, add to open
                if not open.find_item(node) and node not in closed:
                    open.insert(Item(node,f[node]))

        #add curr to closed
        closed.append(curr)
    
 
    return (pred, shortest_path(pred, destination, source))

#test


g = WeightedGraph(4)

x = {}
y= {}

for n in range(len(g.graph)):
    x[n] = n
    y[n] = 2*n


hu = heuristic(g,3,x,y)

A_Star(g,2,3,hu)


#test 2

pq = MinHeap([])

for i in range(0,3):
    item = Item(i,3*i)

    pq.insert(item)

#test

wg = WeightedGraph(7)

wg.add_edge(0,1,1.5)
wg.add_edge(0,2,3)
wg.add_edge(1,2,1)
wg.add_edge(2,3,7)
wg.add_edge(2,4,5)
wg.add_edge(2,6,2)
wg.add_edge(4,5,6)
wg.add_edge(6,5,1)

print (wg.graph)

hue = {}

for i in range(0,7):
    hue[i] = 0

print("h=",hue)

print("A star=",A_Star(wg,0,5,hue))

print("weight=",wg.weights)

KeyError: 3

In [8]:
1 in {1:'a', 2:'b'}

True